<a href="https://colab.research.google.com/github/KunavazhagannSridar/Telium/blob/main/Telium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Telium Version 1.8**

In [ ]:
# @title
import sys
import time
import random
from IPython.display import clear_output

# Global variables

num_modules = 21
module = 1
last_module = 0
possible_moves = []
alive = True
won = False
killed = False
power = 1000
fuel = 500
locked = 0
queen = 0
people = 4
vent_shafts = []
info_panels = []
workers = []
fuel_cells = []
mod_type = ''
action = ''
move = 1
diff = 0

# Procedure declarations

def write(txt):
  for i in txt:
    print(i, end = '', flush = True)
    time.sleep(0.01)
  print()

def thing(txt):
  for i in txt:
    if i == ' ':
      print(i, end = '', flush = True)
    else:
      print(i, end = '', flush = True)
      time.sleep(0.05)
  print()

def load_module():
  global module, possible_moves
  possible_moves = get_modules_from(module)
  output_module()

def get_modules_from(module):
  global mod_type
  moves = []
  text_file = open('module' + str(module) + '.txt', 'r')
  mod_type = text_file.readline()
  for counter in range(0, 4):
    move_read = text_file.readline()
    move_read = int(move_read.strip())
    if move_read != 0:
      moves.append(move_read)
  text_file.close()
  return moves

def output_module():
  global module, mod_type
  print()
  print('--------------------------------------------------------------')
  print()
  message = 'You are in module ' + str(module) +  ' - This is the ' +  str(mod_type)
  write(message)
  check_npcs()
  print()

def check_npcs():
  global module, fuel, queen, vent_shafts, fuel_cells, workers, info_panels, power
  if module == queen:
    move_queen()
  elif module in vent_shafts:
    write('There is a bank of fuel cells in here.')
    write('You load one into your flame thrower.')
    fuel_gained = random.randint(2, 5) * 10
    fuel = fuel + fuel_gained
    message = 'Fuel now reading ' + str(fuel)
    write(message)
    write('The doors suddenly lock.')
    write('You climb into the ventilation shaft.')
    write('You follow the passage and find yourself sliding into a module.')
    clear_output()
    design2()
    last_module = module
    module = random.randint(1, num_modules)
    while module == last_module or module in vent_shafts:
      module = random.randint(1, num_modules)
    load_module()
  elif module in info_panels:
    write('There is an information panel in this module.')
    print('1. Interact')
    print('2. Ignore')
    choice = 0
    while choice not in [1, 2]:
      try:
        choice = int(input('> '))
      except ValueError:
        write('Invalid')
    if power > 50:
      if choice == 1:
        message = 'The queen is in module ' + str(queen)
        write(message)
        power -= 50
    else:
      if choice == 1:
        write('There is not enough power. The panel is dead.')
  elif module in workers:
    worker_aliens()
  elif module in fuel_cells:
    write('There is a fuel cell in this module.')
    write('You load it into your flamethrower.')
    fuel_gained = random.randint(2, 5) * 10
    fuel = fuel + fuel_gained
    message = 'You now have ' + str(fuel) + ' fuel.'
    write(message)

def output_moves():
  global possible_moves
  print()
  write('From here you can move to modules: ')
  for move in possible_moves:
    message = str(move) + ' '
    write(message)
  print()

def lock():
  global num_modules, power, locked
  valid = False
  while not valid:
    try:
      new_lock = int(input('Enter module to lock: '))
    except ValueError:
      write('Invalid')
    else:
      valid = True
  if new_lock < 0 or new_lock > num_modules:
    write('Invalid module. Operation failed.')
  elif new_lock == queen:
    write('Operation failed. Unable to lock module.')
  elif new_lock == locked:
    write('Module already locked.')
  else:
    locked = new_lock
    message = 'Aliens cannot get into module ' + str(locked)
    write(message)
  power_used = 25 + 5 * random.randint(0, 5)
  power -= power_used
  input()

def move_queen():
  global num_modules, module, last_module, locked, queen, won, vent_shafts, killed, people, fuel
  if module == queen:
    write('There it is! The queen alien is in this module...')
    moves_to_make = random.randint(1, 3)
    can_move_to_last_module = False
    while moves_to_make > 0:
      escapes = get_modules_from(queen)
      if module in escapes:
        escapes.remove(module)
      if last_module in escapes and can_move_to_last_module == False:
        escapes.remove(last_module)
      if locked in escapes:
        escapes.remove(locked)
      if len(escapes) == 0:
        moves_to_make = 0
        write("...and the door is locked. It's trapped.")
        if fuel >= 100:
          won = True
        else:
          people -= 1
          if people <= 0:
            killed = True
            return
          write('The queen has killed an astonaut.')
          write('You have retreated.')
          if last_module in vent_shafts:
            module = possible_moves[random.randint(0, 3)]
          else:
            module = last_module
          fuel = 250
          input()
          clear_output()
          load_module()
      else:
        if moves_to_make == 1:
          write("...and has escaped.")
        queen = random.choice(escapes)
        moves_to_make -= 1
        can_move_to_last_module = True
        while queen in vent_shafts:
          if moves_to_make > 1:
            write("...and has escaped.")
          write('We can hear scuttling in the vent shafts.')
          valid_move = False
          while not valid_move:
            valid_move = True
            queen = random.randint(1, num_modules)
            if queen in vent_shafts:
              valid_move = False
          moves_to_make = 0

def worker_aliens():
  global module, workers, fuel, alive, vent_shafts, last_module, possible_moves, people, diff
  if module in workers:
    write('Startled, a young alien scuttles across the floor.')
    write('It turns and leaps towards us.')
    successful_attack = False
    while not successful_attack:
      print()
      print('1. Short burst to frighten it')
      print('2. Long burst to kill it')
      print('3. Run away')
      action = 0
      while action not in [1, 2, 3]:
        try:
          action = int(input('> '))
        except ValueError:
          write('Invalid')
      if action == 3:
        is_killed = random.randint(0, diff)
        if is_killed == 1:
          people -= 1
          if people <= 0:
            killed = True
            return
          write('The alien has killed one astronaut.')
          input()
        if last_module in vent_shafts:
          module = possible_moves[random.randint(0, 3)]
        else:
          module = last_module
        fuel = 250
        clear_output()
        load_module()
        return
      try:
        fuel_used = int(input('How much fuel will you use: '))
      except ValueError:
        write('Error, no fuel used.')
        fuel_used = 0
      fuel = fuel - fuel_used
      if fuel <= 0:
        write('Not enough fuel.')
        people -= 1
        print(people)
        if people <= 0:
          killed = True
          return
        write('An astronaut has been killed.')
      elif action == 1:
        fuel_needed = 30 + 10*random.randint(0, 5)
      elif action == 2:
        fuel_needed = 90 + 10*random.randint(0, 5)
      if fuel_used >= fuel_needed:
        successful_attack = True
      else:
        write('The alien squeals but it is not dead. It is angry.')
    if action == 1:
      write('The alien scuttles away into a corner of the room.')
    elif action == 2:
      write('The alien has been vaporised.')
      workers.remove(module)
    input()

def intuition():
  global possible_moves, workers, vent_shafts, queen, info_panels
  sound = False
  feel = False
  near_panel = False
  for connected_module in possible_moves:
    if connected_module in workers:
      sound = True
    if connected_module in vent_shafts:
      feel = True
    if connected_module == queen:
      write('Listen...Did you hear that?')
    if connected_module in info_panels:
      near_panel = True
  if feel == True:
    write('We can feel cold air.')
  if sound == True:
    write('We can hear scuttling.')
  if near_panel == True:
    write('There should be a panel near here. We could use it to find the queen.')

def get_action():
  global module, last_module, possible_moves, action, move, power, vent_shafts, queen, workers, info_panels, fuel_cells
  print('1. Move')
  print('2. Use scanner')
  choice = 0
  while choice not in [1, 2]:
    try:
      choice = int(input('> '))
    except ValueError:
      write('Choice invalid')
  if choice == 1:
    valid = False
    move = 0
    while move not in possible_moves:
      try:
        move = int(input('Enter module to move to: '))
      except ValueError:
        move = int(input('Enter module to move to: '))
    last_module = module
    module = move
  else:
    print()
    print('1. Power')
    print('2. Lock')
    print('3. Lifeforms')
    print('4. Scan')
    option = 0
    while option not in [1, 2, 3, 4]:
      try:
        option = int(input('> '))
      except ValueError:
        write('Choice invalid')
    if option == 1:
      message = 'Power level: ' + str(power)
      write(message)
      input()
    elif option == 2:
      write('Scanner ready. Press enter to lock module.')
      input()
      lock()
    elif option == 3:
      lifeforms = len(workers) + 1
      message = 'Lifeforms detected: ' + str(lifeforms)
      write(message)
      input()
    elif option == 4:
      if power < 26:
        write('Not enough power left.')
      else:
        scan_module = 0
        while scan_module not in possible_moves:
          try:
            scan_module = int(input('Enter module to scan: '))
          except ValueError:
            write('Invalid')
        if scan_module in vent_shafts:
          write('There is a vent shaft in this module.')
        if scan_module in workers:
          write('There is a worker in this module.')
        if scan_module in info_panels:
          write('There is an info panel in this module.')
        if scan_module in fuel_cells:
          write('There is a fuel cell in this module.')
        if scan_module == queen:
          write('The queen is in this module.')
        else:
          write('There is nothing special in this module.')
        input()

def spawn_npcs():
  global num_modules, queen, vent_shafts, info_panels, workers, fuel_cells, diff
  module_set = []
  for counter in range(2, num_modules + 1):
    module_set.append(counter)
  random.shuffle(module_set)
  i = 0
  queen = module_set[i]
  for counter in range(0, 5 - diff):
    i = i + 1
    vent_shafts.append(module_set[i])
  for counter in range(0, 5 - diff):
    i = i + 1
    info_panels.append(module_set[i])
  for counter in range(0, 5 - diff):
    i = i + 1
    workers.append(module_set[i])
  for counter in range(0, 5 - diff):
    i = i + 1
    fuel_cells.append(module_set[i])

def play_game():
  global power, alive, won, killed
  spawn_npcs()
  while alive and not won:
    if killed == True:
      break
    load_module()
    power -= 1
    if power == 0:
      alive == False
    elif people <= 0:
      killed = True
    if not won and alive:
      if killed == True:
        break
      if power < 101:
        write('Life support will shut off any minute now.')
      if fuel <= 99:
        write('Low fuel!')
      intuition()
      output_moves()
      get_action()
      clear_output()
      design2()
  if killed == True:
    write('The alien lunges at you and kills you.')
    write('Game over. You lost!')
    input('Press enter to return to menu.')
  elif won == True:
    write('The queen is trapped and you burn it to death with your flamethrower.')
    write('Game over. You won!')
    input('Press enter to return to menu.')
  elif alive == False:
    write('The station has run out of power. Unable to sustain life support, you die.')
    write('Game over. You lost!')
    input('Press enter to return to menu.')

def output_story():
  clear_output()
  design2()
  write('A bacterial species was discoverd on Mars and brought to the space station Charles Darwin for testing.')
  write('After incubating it in conditions similiar to ancient Mars, the scientists discovered it began to grow.')
  write('It grew by feeding on the other bacteria and day by day its size and strength grew.')
  write('Two days ago the alien killed a scientist and escaped into the depths of the Charles Darwin.')
  write('You are now trapped on the Charles Darwin with a dangerous alien.')
  write('The alien damaged the life support system. After a while the astronauts wil die.')
  write('Help will only arrive when the alien is killed.')
  write('Time is running out...')
  input('Press enter to continue... ')
  clear_output()
  design2()

def instructions():
  global diff
  clear_output()
  design2()
  write('The alien must be trapped in a module and killed with a flamethrower before life support runs out.')
  write(f'You have {power} power to start with. Every turn this will decrease by 1 and when it reaches 0, you die.')
  write(f'There are {diff} astornauts. If all of them die then you lose.')
  write("Using your scanner or information panels also depletes the station's power.")
  write('You can move from module to module to find the alien. You can also lock modules to trap the alien or use a scanner to find it.')
  write('If you enter a module with a vent the doors will lock and you will have to take the vent to a random module.')
  write('You may encounter a worker aliens. You can choose to frighten them or kill them using your flamethrower.')
  write('If you fail to kill/frighten them they will kill you.')
  write(f'You start with {fuel} fuel.')
  write('You will be able to pick up fuel for your flamethrower in certain modules.')
  write('Good luck, astronaut!')
  input('Press enter to continue... ')
  clear_output()
  design2()

def design1():
  thing('-------  |------  |       ------- |        | |\\         /|')
  time.sleep(0.5)
  thing('   |     |        |          |    |        | | \\       / |')
  time.sleep(0.5)
  thing('   |     |------  |          |    |        | |  \\     /  |')
  time.sleep(0.5)
  thing('   |     |        |          |    |        | |   \\   /   |')
  time.sleep(0.5)
  thing('   |     |------  |------ ------- \\--------/ |    \\ /    |')
  print()
  time.sleep(0.5)

def design2():
  print('-------  |------  |       ------- |        | |\\         /|')
  print('   |     |        |          |    |        | | \\       / |')
  print('   |     |------  |          |    |        | |  \\     /  |')
  print('   |     |        |          |    |        | |   \\   /   |')
  print('   |     |------  |------ ------- \\--------/ |    \\ /    |')
  print()

def output_diff():
  global diff
  if diff == 1:
    print('1. Beginner')
    print('2. Explorer')
    print('3. Astronaut')
    print('4. Soldier (Current)')
  elif diff == 2:
    print('1. Beginner')
    print('2. Explorer')
    print('3. Astronaut (Current)')
    print('4. Soldier')
  elif diff == 3:
    print('1. Beginner')
    print('2. Explorer (Current)')
    print('3. Astronaut')
    print('4. Soldier')
  elif diff == 4:
    print('1. Beginner (Current)')
    print('2. Explorer')
    print('3. Astronaut')
    print('4. Soldier')
  else:
    print('1. Beginner')
    print('2. Explorer')
    print('3. Astronaut')
    print('4. Soldier')

def select_diff():
  global diff, fuel, power, people
  output_diff()
  thingy = 0
  while thingy not in [1, 2, 3, 4]:
    try:
      thingy = int(input('Select difficulty: '))
    except ValueError:
      write('Select a valid option.')
  diff = 5 - thingy
  fuel = diff * 100
  power = diff * 100 + 500
  people = diff
  print()

# Main program

design1()
select_diff()
clear_output()
design2()
while True:
  print('1. Play')
  print('2. Story')
  print('3. Instructions')
  print('4. Change difficulty')
  print('5. Quit')
  option = 0
  while option not in [1, 2, 3, 4, 5]:
    try:
      option = int(input('> '))
    except ValueError:
      print('Select a valid option.')
  if option == 1:
    clear_output()
    design2()
    play_game()
    clear_output()
    design2()
  elif option == 2:
    output_story()
    clear_output()
    design2()
  elif option == 3:
    instructions()
    clear_output()
    design2()
  elif option == 4:
    clear_output()
    design2()
    select_diff()
    clear_output()
    design2()
  else:
    break
clear_output()
design2()